In [ ]:
import numpy as np
from numpy.random import seed
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
from collections import defaultdict
import random
import os
import joblib

In [2]:
from sklearn import metrics
from sklearn.metrics import *
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold

In [3]:
import keras
from keras import utils as np_utils
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier

In [4]:
from sklearn.ensemble import RandomForestClassifier 
from sklearn.inspection import permutation_importance
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)



In [5]:
def reset_seed(seed):
    random.seed(seed) 
    np.random.seed(seed) 
    os.environ["PYTHONHASHSEED"] = str(seed)
    os.environ["CUDA_VISIBLE_DEVICES"] = '1'
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    os.environ['TF_CUDNN_USE_FRONTEND '] = '1'
    tf.random.set_seed(seed)

# Data Load

In [6]:
def getdata_random(fn, sample=True):
    
    #negative set
    df0 = pd.read_csv('../random2.out', header=None, names=['seq'])
    df0['label'] = 0
    
    #positive set
    df1 = pd.read_csv(fn, header=None, names=['seq','cnt'])
    df1['label'] = 1
    
    if sample==True:
        df1=df1.sample(frac=1, random_state=seed_num).reset_index(drop=True)
        df0=df0.sample(frac=1, random_state=seed_num).reset_index(drop=True)
        df1.drop('cnt', axis=1, inplace=True)
        df = pd.concat([df1[:10000], df0[:10000]])
    else:
        #cnt가 3 이상인 데이터만 추출
        df1 = df1[df1.cnt>=3]
        
        # Train, Test set에 cnt값은 필요 없기 때문에 제거
        df1.drop('cnt', axis=1, inplace=True)

        # [:sample_size]
        df = pd.concat([df1[:10000], df0[:10000]])
        
    df = df.reset_index(drop=True)
    return df

In [7]:
bases=['A','C','G','T']

def dnatoidx(k):

    dtoi = {}
    kmers=[''.join(p) for p in itertools.product(bases, repeat=k)]
    idx = 0
    for kmer in kmers:
        dtoi[kmer]=idx
        idx += 1
    return dtoi

def count_kmer(seq, dtoi, k):
    kmercnt = [0]*len(dtoi)
    for i in range(30-k+1):
        kmer = seq[i:i+k]
        kmercnt[dtoi[kmer]] += 1
    return kmercnt

def count_starkmer(seq, dtoi, k, blank):
    kmercnt = [0]*len(dtoi)
    for i in range(len(seq)-(k+blank)+1):
        kmer = seq[i]+seq[i+k+blank-1]
#         print(i, kmer)
        kmercnt[dtoi[kmer]] += 1
    return kmercnt

# MLP

In [9]:
def MLP_model(k):
    reset_seed(seed_num)
    MLP_model = Sequential([
        Dense(100, activation='relu',input_shape=(4**k,)),
        Dropout(0.1, seed=seed_num),
        Dense(30, activation='relu'),
        Dropout(0.1, seed=seed_num),
        Dense(2, activation = 'softmax')
    ])
    MLP_model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy','AUC'])
    return MLP_model

# One-hot Encoding

In [10]:
def preprocessing(df):
    encoded_list = []

    def encode_seq(s):
        Encode = {'A':[1,0,0,0],'T':[0,1,0,0],'C':[0,0,1,0],'G':[0,0,0,1], 'N':[0,0,0,0]}
        return [Encode[x] for x in s]

    for i in df.seq:
        x = encode_seq(i)
        encoded_list.append(x)

    X = np.array(encoded_list)
    X = X.astype('float32')
    X.shape
    
    return X

# Train, test split

In [11]:
def data_split(df,seed):
    dataX= preprocessing(df)
    dataY=df.label
    X_train,X_test,Y_train,Y_test=train_test_split(dataX,dataY,test_size=0.2,random_state=seed)

    Y_train = keras.utils.np_utils.to_categorical(Y_train)
    Y_test = keras.utils.np_utils.to_categorical(Y_test)

    return X_train,X_test,Y_train,Y_test

# Model Evaluation

In [12]:
def make_plot(name,history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{name} loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [13]:
def model_evaluation(model, X_test, Y_test,verbose=0):
    test_prediction = model.predict(X_test,verbose = 0)
    prediction = np.argmax(test_prediction, axis=1)
    prediction2 = test_prediction.T[1].T
    answer = np.argmax(Y_test, axis=1)
    confusion = confusion_matrix(answer, prediction)
    
    
    acc = accuracy_score(answer, prediction)
    
    
    auc  = roc_auc_score(answer, prediction2)
    
    
    pre = precision_score(answer, prediction)
    

    recall = recall_score(answer, prediction)
    

    specificity = confusion[0,0]/(confusion[0,1]+confusion[0,0])
    

    f1 = f1_score(answer, prediction)
    
    
    
    if verbose == 1:
        print('\nTest confusion matrix :')
        print(pd.DataFrame(confusion))
        print('\nTest Accuracy score: ',acc)
        print('Test AUC score: ', auc)
        print('Test Precision score: ', pre)
        print('Test Recall score: ', recall)
        print('Test Specificity score: ', specificity)
        print('Test F1_score: ', f1,'\n')
    
    return acc , auc, pre, recall, specificity, f1, answer, prediction2

In [14]:
def RF_evaluation(model, X_test, Y_test,verbose=0):
    prediction=model.predict(X_test)
    prediction2=model.predict_proba(X_test).T[1].T
    
    answer = np.argmax(Y_test, axis=1)
    confusion = confusion_matrix(answer, prediction)
    
    
    acc = accuracy_score(answer, prediction)
    
    
    auc  = roc_auc_score(answer, prediction2)
    
    
    pre = precision_score(answer, prediction)
    

    recall = recall_score(answer, prediction)
    

    specificity = confusion[0,0]/(confusion[0,1]+confusion[0,0])
    

    f1 = f1_score(answer, prediction)
    
    
    
    if verbose == 1:
        print('\nTest confusion matrix :')
        print(pd.DataFrame(confusion))
        print('\nTest Accuracy score: ',acc)
        print('Test AUC score: ', auc)
        print('Test Precision score: ', pre)
        print('Test Recall score: ', recall)
        print('Test Specificity score: ', specificity)
        print('Test F1_score: ', f1,'\n')
    
    return acc , auc, pre, recall, specificity, f1, answer, prediction2

# Model Train

In [15]:
def MLP_kmer_train(data,df,seed):
    reset_seed(seed_num)
    
    call_back = tf.keras.callbacks.ModelCheckpoint(
                f'R{r}_MLP_{k}_mer_{itr}.h5', monitor='val_loss', verbose=0, save_best_only=True,
                save_weights_only=False, mode='auto', save_freq='epoch', options=None
                )

    MLP=MLP_model(k)
    history = MLP.fit(data[0], data[1], batch_size = 1024, 
                                epochs=epochs, validation_data=(data[2],data[3]), 
                                shuffle=True,callbacks=[call_back],
                                verbose = 0)
    
    make_plot(f'mlp_{k}-mer',history)
    best_model = tf.keras.models.load_model(f'R{r}_MLP_{k}_mer_{itr}.h5')
    
    acc , auc, pre, recall, specificity, f1,answer,pred = model_evaluation(best_model, data[4],data[5],verbose = 1)
    
    df.loc[len(df)] = [f'mlp-{k}mer',r,0 , acc , auc, pre, recall, f1]

    
    return best_model, df, answer, pred

In [16]:
def RF_kmer_train(data,df,seed):
    reset_seed(seed_num)
    RF=RandomForestClassifier(random_state=itr)
    RF.fit(X_train,y_train)
    acc , auc, pre, recall, specificity, f1,answer,pred = RF_evaluation(RF, data[4],data[5],verbose = 1)
    df.loc[len(df)] = [f'RF-{k}mer',r,0 , acc , auc, pre, recall, f1]
    joblib.dump(RF,f'./R{r}_RF_{k}mer_{itr}.pkl')
    return RF, df, answer, pred

# Results

In [ ]:
epochs = 60
kmer_df = pd.DataFrame(columns=["model","round","shuffle","acc","auc","pre","recall","f1"])

for k in [1,2,3,4,5]:


    print(k)
    scores = defaultdict(list)
    dtoi = dnatoidx(k)
    n_seq = 10000

    for r in [3,4,5,6]:
        df = getdata_random(f'../data/R{r}_1.out_N_unique', False)

        cntfeature = []
        for seq in df.seq:
            cntfeature.append( count_kmer(seq, dtoi, k) )
            
        X = np.array(cntfeature)
        y = df.label

        for itr in range(10):
            seed_num = itr
            reset_seed(seed_num)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,random_state=seed_num)
            X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5,random_state=seed_num)

            y_train2 = keras.utils.np_utils.to_categorical(y_train)
            y_valid2 = keras.utils.np_utils.to_categorical(y_valid)
            y_test2 = keras.utils.np_utils.to_categorical(y_test) 
            
            data = [X_train,y_train2,X_valid,y_valid2,X_test,y_test2]
            
            reset_seed(seed_num)
            model ,kmer_df,_,_ = MLP_kmer_train(data,kmer_df,seed_num)
            reset_seed(seed_num)
            model ,kmer_df,_,_ = RF_kmer_train(data,kmer_df,seed)
        print(kmer_df)

In [47]:
kmer_df

,model,round,shuffle,acc,auc,pre,recall,f1
0,mlp-1mer,3,0,0.67250,0.734149,0.655686,0.726500,0.689279
1,RF-1mer,3,0,0.65100,0.693786,0.640074,0.690000,0.664100
2,mlp-1mer,3,0,0.65825,0.710449,0.647241,0.720747,0.682019
3,RF-1mer,3,0,0.62225,0.668306,0.628249,0.629794,0.629020
4,mlp-1mer,3,0,0.66875,0.722533,0.644725,0.726995,0.683393
...,...,...,...,...,...,...,...,...
395,RF-5mer,6,0,0.80200,0.882307,0.765736,0.844909,0.803376
396,mlp-5mer,6,0,0.85875,0.920777,0.846483,0.880376,0.863097
397,RF-5mer,6,0,0.81225,0.889381,0.799717,0.838853,0.818818
398,mlp-5mer,6,0,0.84875,0.914672,0.839086,0.851623,0.845308


In [48]:
#kmer_df.to_csv('k-mer_all.csv',index=False)